<a href="https://colab.research.google.com/github/arulrajgopal-zerotoone/zero_to_one_spark/blob/main/apache_spark/14_handle_nested_schema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

from pyspark.sql import SparkSession

#create spark session
spark= SparkSession.builder.appName('mysparksession').getOrCreate()

#create spark context
sc = spark.sparkContext

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import concat, col,lit

In [5]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)

])
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)

])

In [10]:
drivers_df = spark.read \
    .format("json")\
    .schema(drivers_schema)\
    .load('formula1_drivers.json')


drivers_df.show()

+--------+----------+------+----+--------------------+----------+-----------+--------------------+
|driverId| driverRef|number|code|                name|       dob|nationality|                 url|
+--------+----------+------+----+--------------------+----------+-----------+--------------------+
|       1|  hamilton|    44| HAM|   {Lewis, Hamilton}|1985-01-07|    British|http://en.wikiped...|
|       2|  heidfeld|  NULL| HEI|    {Nick, Heidfeld}|1977-05-10|     German|http://en.wikiped...|
|       3|   rosberg|     6| ROS|     {Nico, Rosberg}|1985-06-27|     German|http://en.wikiped...|
|       4|    alonso|    14| ALO|  {Fernando, Alonso}|1981-07-29|    Spanish|http://en.wikiped...|
|       5|kovalainen|  NULL| KOV|{Heikki, Kovalainen}|1981-10-19|    Finnish|http://en.wikiped...|
|       6|  nakajima|  NULL| NAK|  {Kazuki, Nakajima}|1985-01-11|   Japanese|http://en.wikiped...|
|       7|  bourdais|  NULL| BOU|{Sébastien, Bourd...|1979-02-28|     French|http://en.wikiped...|
|       8|

In [13]:
nest_removed_df = drivers_df.withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))
nest_removed_df.show()

+--------+----------+------+----+------------------+----------+-----------+--------------------+
|driverId| driverRef|number|code|              name|       dob|nationality|                 url|
+--------+----------+------+----+------------------+----------+-----------+--------------------+
|       1|  hamilton|    44| HAM|    Lewis Hamilton|1985-01-07|    British|http://en.wikiped...|
|       2|  heidfeld|  NULL| HEI|     Nick Heidfeld|1977-05-10|     German|http://en.wikiped...|
|       3|   rosberg|     6| ROS|      Nico Rosberg|1985-06-27|     German|http://en.wikiped...|
|       4|    alonso|    14| ALO|   Fernando Alonso|1981-07-29|    Spanish|http://en.wikiped...|
|       5|kovalainen|  NULL| KOV| Heikki Kovalainen|1981-10-19|    Finnish|http://en.wikiped...|
|       6|  nakajima|  NULL| NAK|   Kazuki Nakajima|1985-01-11|   Japanese|http://en.wikiped...|
|       7|  bourdais|  NULL| BOU|Sébastien Bourdais|1979-02-28|     French|http://en.wikiped...|
|       8| raikkonen|     7| R